# INF8111 - Fouille de données


## TP2 Automne 2019 - Extraction et analyse d'une base de données de tweets

##### Membres de l'équipe:

    - Boisvenue, Étienne (1798942)
    - Le Page, Pierre-Étienne (1689985)
    - Wong, Jade (1738089)

## Présentation du problème

En 2017, Twitter compte 313 millions d’utilisateurs actifs par mois avec 500 millions de tweets envoyés par jour. Cette information est rendue disponible à destination de la recherche et du développement web grâce à une API publique qui permet de collecter les informations que l'on souhaite.

Néanmoins, la politique de développement de Twitter limite le partage de ces données. En effet, le partage du contenu des tweets dans une base de données n'est pas autorisé, seuls les identifiants des tweets le sont. 
Pour partager publiquement une base de données de tweets que l'on a créée, il faut que cette base de données ne soit consituée que des identifiants de tweets, et c'est ce que l'on retrouve dans la plupart des jeux de données publiques.

Il est donc nécessaire pour exploiter ces données "d'hydrater" les tweets en question, c'est-à-dire extraire l'ensemble des informations à partir de l'ID, ce qui demande d'utiliser l'API de Twitter.

Nous allons ici utiliser des bases de données publiques créées par GWU (George Washington University), qui ont l'avantage d'être très récentes : 
https://dataverse.harvard.edu/dataverse/gwu-libraries

Chaque base de données de GWU couvre un sujet précis (élection américaine de 2016, jeux olympiques, etc.), et les données ont été recueillis en appliquant des requêtes qui filtraient les résultats pour n'avoir que des tweets pertinents. Un fichier README est fourni avec chaque base de données pour donner les détails de création du *dataset*. 


**Les objectifs de ce TP sont donc les suivants :**

 1. Construire un *crawler* qui collecte les informations d'un tweet à partir de son ID, avec le jeu de données de son choix et les informations pertinentes pour le sujet choisi
 2. A partir de ces données de Twitter collectés, application de méthodes en Machine Learning (ML)/Natural Language Processing (NLP) pour fournir une analyse pertinente. 


Twitter autorisant le partage **local** des données (par exemple au sein d'un groupe de recherche), une base de données sera fournie si vous ne parvenez pas à créer la vôtre.

# I/ Hydratation de tweets à l'aide de l'API Twitter (4 Pts)

### 1. Obtenir l'authorisation de Twitter pour l'utilisation de l'API

Pour l'authentification, Twitter utilise OAuth : https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
Vous aurez ici besoin en particulier de OAuth2, car vous n'allez pas interagir avec des utilisateurs sur Twitter (simplement collectés des données).

##### 1.1. Obtention d'un compte Twitter développeur

 La première étape nécessaire pour enregistrer votre application et de créer un compte Twitter développeur. Pour ce faire :

 - Créez un compte Twitter classique si vous n'en avez pas déjà un.
 
 - Sur le site, https://developer.twitter.com, cliquez sur *apply* pour obtenir un compte développeur. 
 
 - Remplissez tous les champs nécessaires. Twitter demande beaucoup de détails sur l'utilisation que vous allez faire de ce compte, il est donc important d'expliquer la démarche en détail : il faut souligner le fait que le projet est **académique** (aucune intention commerciale, aucune publication des données collectés, etc.), expliquer les objectifs et l'apprentissage de ce TP (prise en main de l'API Twitter, l'application concrète de méthodes de Data Mining, etc.), mais aussi expliquer en détail ce que vous allez faire des données (en reprenant des consignes du sujet), les méthodes que vous allez appliquer (citez des méthodes vues en cours ou au précédent TP), le rendu fourni (insistez sur le fait que rien ne sera publique), etc. Pensez notamment à indiquer le nom du cours et le sigle du cours, le nom de l'établissement, mon nom (Théo Moins), etc. Cochez que vous n'utiliserez pas la fonctionnalité de Retweet, et que l'aggregation et l'affichage de tweets ne sera fait que dans un cadre pédagogique (non publique, et sous la forme d'un projet de recherche). Si jamais vous n'êtes pas assez précis, Twitter peut vous renvoyer un courriel pour vous demander des précisions. 

##### 1.2. Obtention d'un jeton d'accès

 - Lorsque Twitter aura validé votre demande de compte développeur, allez sur https://developer.twitter.com/en/apps pour créer une application (cliquer sur *create an app*)

- Ici encore, des informations sont à fournir ici. Certaines, comme le nom ou le site internet, ne sont pas très importante, vous pouvez mettre un site internet factice si vous le souhaitez.

- A la fin de ce processus, vous pouvez enfin obtenir les clés et les jetons pour utiliser l'API: allez sur la page de l'application pour créer les jetons. Vous devez récupérer une paire de clés et une paire de jetons pour passer à la suite.



In [1]:
CONSUMER_KEY = "EpLoTTNfe43QhKjhflj1COQnu"
CONSUMER_SECRET = "xtEuo845wmH8iBhboP0MQxM4DxznCYg2tO91MJN9KhFTrSl9cx"

oauth_token = "1181907703695904768-8QJXBucdMidyASdDvlNZrh2Kt4TcP4"
oauth_secret = "Zoy3gnUSNfRCdmmFyAyNmDtzlfF2M4rfIt8tpxWVm3V0t"

###  2. Premiers pas avec Twython

##### 2.1 Installation et import de la librairie


Plusieurs librairies Python existent pour manipuler l'API Twitter. Aussi appelé *wrappers*, ce sont un ensemble de fonctions python qui appelle des fonctions de l'API. Parmi elles, nous utiliserons Twython, librairie répendue et activement maintenue.

Documentation de Twython : https://twython.readthedocs.io/en/latest/api.html 

In [2]:
import csv
import time
import sys

try:
    from twython import Twython, TwythonError, TwythonRateLimitError
except ImportError:
    !pip install --user twython

##### 2.2 Création d'une application et premiers tests:

In [3]:
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, oauth_token, oauth_secret)

Voici un test avec une recherche très simple pour vous assurer que la requête fonctionne.

La fonction search renvoie une recherche (non exhaustive) de tweets, et l'option "*popular*" permet de retourner les résultats les plus populaires de la réponse. (documentation ici: https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets)

In [4]:
basic_search = twitter.search(q='python', result_type='popular')

La fonction `search` renvoie un dictionnaire contenant la liste de tweets de la requête, et les métadonnées.

Voici un exemple d'un résultat d'une recherche, observez ainsi toutes les données/métadonnées que contient un tweet et que vous pouvez extraire par la suite:

In [5]:
basic_search['statuses'][0]

{'created_at': 'Sat Nov 09 18:00:08 +0000 2019',
 'id': 1193226783342157826,
 'id_str': '1193226783342157826',
 'text': 'Python took the silver from Java this year, becoming the #2 most popular programming language on GitHub by reposito… https://t.co/vfo8dRkCjc',
 'truncated': True,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/vfo8dRkCjc',
    'expanded_url': 'https://twitter.com/i/web/status/1193226783342157826',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="https://sproutsocial.com" rel="nofollow">Sprout Social</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 13334762,
  'id_str': '13334762',
  'name': 'GitHub',
  'screen_name': 'github',
  'location': 'San Francisco, C

Il est également possible avec Twython de récupérer les informations d'un tweet à partir de son ID. 

#### Question 1. Afficher la date, le nom d'utilisateur et le contenu du tweet ayant l'ID : 1157345692517634049 (0.5 Pts)

*Indice : vous pourrez utiliser avec la fonction de twython `show_status`*

In [6]:
# Identifiant unique représentant le tweet
test_id = "1157345692517634049"


# Extraction de toutes les données de tweet (incluant son contenu)
tweet = twitter.show_status(id=test_id)

# Toutes les données accessibles
print('\n Ensemble des données accessibles à propos du tweet:')
print(tweet.keys())

# Extraction de la date
date=tweet['created_at']
print('\n Date:')
print(date)

# Extraction du nom de l'utilisateur 
nom=tweet['user']['name']
print('\n Auteur:')
print(nom)

# Extraction du screen name de l'utilisateur
screen_name=tweet['user']['screen_name']
print("\n Screen name de l'utilisateur:")
print(nom)

# Extraction du contenu du tweet
contenu=tweet['text']
print('\n Contenu du tweet:')
print(contenu)



 Ensemble des données accessibles à propos du tweet:
dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

 Date:
Fri Aug 02 17:41:30 +0000 2019

 Auteur:
Donald J. Trump

 Screen name de l'utilisateur:
Donald J. Trump

 Contenu du tweet:
A$AP Rocky released from prison and on his way home to the United States from Sweden. It was a Rocky Week, get home ASAP A$AP!


**Attention** : Twitter a une limitation de requête par fenêtre de 15 minutes, qui est donc à prendre en compte dans la base de données : https://developer.twitter.com/en/docs/basics/rate-limiting.html

### 3. Hydratation d'une base de donnée de tweets

Les choses sérieuses commencent ! 

On souhaite désormais construire une fonction `hydrate_database` qui, à partir d'un fichier texte contenant une liste d'ID de tweets, créer un fichier csv contenant les informations que l'on souhaite extraire. 

Due à la limitation de requête, la fonction `show_status` vue plus haut s'avère peu efficace pour cette tâche : à raison de 900 requêtes pour 15 minutes, il sera beaucoup trop long de construire une base de données un tant soit peu conséquente. La fonction `lookup_status` (voir documentation) sera donc plus adaptée. Elle permettra d'hydrater 100 tweets par requête, ce qui, a raison d'une limite de 900 requêtes pour 15 minutes, rends la construction de la base de données plus réaliste. Il faudra tout de même gérer l'erreur générer par la limitation, si l'on souhaite avoir plus de 90000 tweets ou si l'on appelle plusieurs fois la fonction en moins de 15 minutes.

#### Question 2. Implémenter la fonction `hydrate_database` (3.5 Pts)

*Attention : Il faut également gérer le cas où la feature demandée n'est pas une clé du dictionnaire mais une "sous-clé", comme c'est le cas pour le nom d'utilisateur par exemple (accessible dans la feature *user*, qui lui même est un dictionnaire). Un moyen simple pour pallier à ce problème consiste à considérer la feature comme une liste, qui contiendrait la clé et les sous-clés si il y a lieu (voir exemple plus bas)

*Indice : La fonction `sleep` du module time permet de patienter le temps nécessaire*

In [13]:
# La fonction permet même d'aller chercher des features qui sont ''imbriquées'' les unes dans les autres
# E.g. le ''screen_name'' d'un ''user''.


def hydrate_database(filename, database_name, 
                     features, nb_requests, 
                     tweet_hydratation_limit=100):
    """
    Create a csv file that contains features of tweets from an file that contains ID of tweets.
    
    filename: Name of the file that contains ids
    database_name: name of the file that will be created
    features: List of features
    nb_requests: number of time the function lookup_status will be called
    tweet_hydratation_limit:
    """
    # Opening the ID File:
    file = open(filename, "r")
    
    # On génère la list de Tweet Ids à envoyer à l'API.
    tweet_id = file.readline()

    lst_tweet_ids = []
    tweet_ids = []

    tweet_ids.append(tweet_id)

    while tweet_id:
        tweet_id = file.readline().rstrip('\r\n')
        tweet_ids.append(tweet_id)

        if len(tweet_ids) == tweet_hydratation_limit:
            lst_tweet_ids.append(",".join(tweet_ids))
            tweet_ids = []
    
    if len(tweet_ids) > 0:
        lst_tweet_ids.append(",".join(tweet_ids))
    
    # Creation of the file that will contain the hydrated tweets:
    with open(database_name, 'w', newline='', encoding="utf-8") as csvfile:  
        
        requestsMade = 0
        
        csvHeader = ''
        # On crée tout d'abord l'entête du csv
        for key in features :
            # Comme toutes les clés sont dans des listes, on ramasse la dernière clé de chaque liste, car ce
            # sont celles qui correspondront aux valeurs retournées.
            if len(key)==1:
                csvHeader += key[-1] + ','
            if len(key)==2:
                csvHeader += key[-2] +'_'+ key[-1] +','
                
            
        
        # On enlève la dernière virgule
        csvHeader = csvHeader[:-1] + '\n'
        
        # On écrit l'entête dans le fichier csv.
        csvfile.write(csvHeader)
        
        for i in range(0, len(lst_tweet_ids)):
            
            # Si on a atteint le nombre de requêtes maximal, on arrête d'itérer
            if requestsMade >= nb_requests:
                break
                
            tweetIds = lst_tweet_ids[i]
                
            try:
                # Effectue la requête à l'API
                tweets = twitter.lookup_status(id=tweetIds)
                
                # Comme la requête n'a pas lancé d'exception, on augmente de 1 le nombre de requêtes effectuées.
                requestsMade += 1
                
                print("Request count : " + str(requestsMade))
                
                #On ramasse le contenu des tweets
                for tweet in tweets:
                    csvLine = ''
                    
                    for key in features :
                        
                        # Utilisé pour aller chercher les sous-propriétés dans la structure du Tweet.
                        valueToFetch = tweet
                        
                        for subKey in key:
                            # S'il manque l'information voulue dans le tweet, on écrit une chaîne vide dans
                            # le csv et on passe au prochain ensemble de clé à rechercher.
                            if subKey not in valueToFetch:
                                valueToFetch = ''
                                break
                            
                            # On descend d'un niveau dans la structure du tweet, jusqu'à ce qu'on arrive à la
                            # sous-clé et valeur recherchée.
                            valueToFetch = valueToFetch[subKey]

                        # Une fois que la valeur associée à la sous-clé a été trouvée, on l'ajoute à la ligne
                        # du csv. On remplace aussi les newlines, les carriage return et les virgules par des
                        # espaces, histoire de s'assurer de l'intégrité du csv.
                        if valueToFetch is not None:
                            csvLine += str(valueToFetch).replace(',', ' ').replace('\r', ' ').replace('\n', ' ') + ','
                        else:
                            csvLine += ','

                    # On enlève la dernière virgule
                    csvLine = csvLine[:-1] + '\n'

                    # On écrit la ligne dans le fichier csv.
                    csvfile.write(csvLine)
                
            except TwythonError as e:
                if isinstance(e, TwythonRateLimitError):
                    retry_after = int(e.retry_after)
                    
                    # On attends le temps prescrit par l'API avant de recommencer.
                    print("We need to wait until : " + str(retry_after))
                    print('Current time:'+str(time.time()))
                    print('This is '+str(retry_after-time.time())+' seconds')
                    
                    while time.time()< retry_after:
                        time.sleep(10)
                        print("We need to wait until : " + str(retry_after))
                        print('Current time:'+str(time.time()))
                    
                    # On décrémente l'index de l'itérateur pour réessayer avec les tweets qu'on a manqués
                    i -= 1
    
    print("Database hydratation complete.")
    
    file.close()


Utilisez le fichier suivant en guise d'example : 
https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/5QCCUU/QPYP8G&version=1.1

On suppose qu'on ne souhaite garder que le texte (*text*) l'ID de l'utilisateur (*user/screen_name*)

In [15]:
filename = "climate_id.txt.00"
database_name = "climate.csv"
features = features = [['id'],
            ['created_at'],
            ['text'],
            ['user','id'],
            ['user','name'],
            ['user','screen_name'],
            ['user','location'],
            ['user','description'],
            ['user','followers_count'],
            ['in_reply_to_status_id'],
            ['lang']]
nb_requests = 1000

hydrate_database(filename, database_name, features, nb_requests, tweet_hydratation_limit=100)

Request count : 1
Request count : 2
Request count : 3
Request count : 4
Request count : 5
Request count : 6
Request count : 7
Request count : 8
Request count : 9
Request count : 10
Request count : 11
Request count : 12
Request count : 13
Request count : 14
Request count : 15
Request count : 16
Request count : 17
Request count : 18
Request count : 19
Request count : 20
Request count : 21
Request count : 22
Request count : 23
Request count : 24
Request count : 25
Request count : 26
Request count : 27
Request count : 28
Request count : 29
Request count : 30
Request count : 31
Request count : 32
Request count : 33
Request count : 34
Request count : 35
Request count : 36
Request count : 37
Request count : 38
Request count : 39
Request count : 40
Request count : 41
Request count : 42
Request count : 43
Request count : 44
Request count : 45
Request count : 46
Request count : 47
Request count : 48
Request count : 49
Request count : 50
Request count : 51
Request count : 52
Request count : 53
Re

Request count : 417
Request count : 418
Request count : 419
Request count : 420
Request count : 421
Request count : 422
Request count : 423
Request count : 424
Request count : 425
Request count : 426
Request count : 427
Request count : 428
Request count : 429
Request count : 430
Request count : 431
Request count : 432
Request count : 433
Request count : 434
Request count : 435
Request count : 436
Request count : 437
Request count : 438
Request count : 439
Request count : 440
Request count : 441
Request count : 442
Request count : 443
Request count : 444
Request count : 445
Request count : 446
Request count : 447
Request count : 448
Request count : 449
Request count : 450
Request count : 451
Request count : 452
Request count : 453
Request count : 454
Request count : 455
Request count : 456
Request count : 457
Request count : 458
Request count : 459
Request count : 460
Request count : 461
Request count : 462
Request count : 463
Request count : 464
Request count : 465
Request count : 466


Request count : 827
Request count : 828
Request count : 829
Request count : 830
Request count : 831
Request count : 832
Request count : 833
Request count : 834
Request count : 835
Request count : 836
Request count : 837
Request count : 838
Request count : 839
Request count : 840
Request count : 841
Request count : 842
Request count : 843
Request count : 844
Request count : 845
Request count : 846
Request count : 847
Request count : 848
Request count : 849
Request count : 850
Request count : 851
Request count : 852
Request count : 853
Request count : 854
Request count : 855
Request count : 856
Request count : 857
Request count : 858
Request count : 859
Request count : 860
Request count : 861
Request count : 862
Request count : 863
Request count : 864
Request count : 865
Request count : 866
Request count : 867
Request count : 868
Request count : 869
Request count : 870
Request count : 871
Request count : 872
Request count : 873
Request count : 874
Request count : 875
Request count : 876


# II/ Analyse d'une base de données au choix (16 pts)

Maintenant que vous êtes en mesure d'hydrater une base de données de tweets efficacement et en prenant en compte les limitations de Twitter, vous pouvez l'appliquer sur le *dataset* qui vous intéresse le plus.

### 1. Instructions

Dans cette partie, vous allez mener **entièrement** de vous-même un projet de *Data Science*, c'est à dire de la collecte des données jusqu'à l'interprétation des résultats. 3 sujets sont proposés, vous devez choisir celui qui vous intéresse le plus parmi :
 
 1. Analyse de sentiments pour la prédiction des résultats de l'élection américaine. 
    
    **Dataset :** "2016 United States Presidential Election Tweet Ids", https://doi.org/10.7910/DVN/PDI7IN  
    
    **Précision :** Ce sujet est assez similaire au TP1 (avec ici sentiment = parti politique), vous êtes donc libre de reprendre ce que vous aviez fait. Cependant, il faudrait aller un peu plus en profondeur ici, par exemple sur l'étape de la classification. De plus, vous avez ici une nouvelle problématique qui est que vos données ne sont pas labellisés (mais la construction des collections devrait vous permettre de labelliser vous-même).
 
 
 2. Détection de discours d'incitation à la haine.
    
    **Dataset :** Modifier votre fonction d'hydratation en utilisant la fonction search pour n'avoir que des tweets récents.
 
     **Précision :** Ce sujet pourrait également être abordé de la même manière que le TP1 : des étapes de preprocessing + de la classification. Néanmoins, dans ce cas, posséder des données avec des labels "incitant à la haine"/"n'insite pas à la haine" est beaucoup plus complexe, car beaucoup de bases de données étiquetés, lors de l'hydratation, se trouveront être quasi-vide, car les tweets auront été supprimés au moment où nous ferons notre requête (car Twitter veille aussi à la suppression de tweets haineux). C'est pourquoi vous êtes obligés de créer une base de données avec des tweets les plus récents possibles, avant qu'ils ne soient potentiellement supprimés. Pour désigner un tweet comme haineux, une méthode serait la détection de vocabulaire haineux, par exemple avec `hatebase.org`, qui propose des larges bases de données très complètes. Vous pouvez créer un compte sur le site pour avoir accès à l'API, et ensuite utiliser cette librairie pour Python : https://github.com/DanielJDufour/hatebase. En modifiant la requête pour n'avoir que des tweets contenant ce vocabulaire, et en le mêlant à de l'analyse de sentiment, vous pourrez obtenir des résultats à analyser. Vous pourriez aussi avoir une approche "utilisateur" pour rechercher des tweets haineux : lorsqu'un tweet est détecter comme haineux, inspecter l'ensemble des tweets de l'utilisateur et/ou de ses *followers*. En bref, beaucoup de possibilités, mais ce sujet est le plus complexe des trois. Je serai donc moins exigeant sur les résultats 'chiffrés', l'important ici étant plus l'analyse, et le fait d'avoir une approche cohérente (il est également très important de prendre le temps de réfléchir à une définition claire de "haineux").


 3. Méthodes de clusterings appliqué au tweet sur l'actualité, et analyse des résultats. 
    
    **Dataset :** "News Outlet Tweet Ids", https://doi.org/10.7910/DVN/2FIFLH

    **Précision :** Application de méthodes de preprocessing, puis de méthodes de clustering pour regrouper les tweets qui mentionnent la même actualité ou catégorie d'actualité (au choix!), puis visualisation, étude en fonction du temps...  Vous devrez trouver quelle est la meilleur méthode de clustering, et celle-ci dépendra de votre approche (nombre de classes connu ? si oui, combien de classes?). 
    
    
Vous êtes entièrement libre sur l'ensemble du processus (choix des informations extraites, méthodes en ML, librairie, etc.). Ici seul les bases de données en elle-même sont rigoureusement imposés. Les précisions faites ici servent juste pour vous guider un peu si vous le souhaitez, mais si vous avez d'autres idées n'hésitez pas ! Ces sujets étant populaires au sein de la communauté scientifique, vous pouvez (**seulement si vous le souhaitez**) vous inspirer d'articles de la littérature, à condition de le citer dans votre rapport et de faire votre propre implémentation. 

#### L'objectif cependant ici n'est pas d'obtenir l'état de l'art, mais d'appliquer une méthodologie claire et rigoureuse que vous aurez construite vous-même. 

Les datasets étant massifs, il est fortement déconseillé de faire une base de données contenant tous les tweets hydratés (par exemple, les auteurs de la BDD n°1 soulignent qu'avec les limitations de l'API cela vous prendrait environ 32 jours). C'est à vous de voir quelle est la taille du dataset dont vous avez besoin.

Pensez aussi à lire le fichier README correspondant à la base que vous avez choisi, afin de vous aider à mieux comprendre vos futurs résultats.

### 2. Rédaction d'un rapport

Pour ce TP, vous allez devoir fournir un rapport qui détail et justifie l'ensemble de votre méthode, et qui fournisse les résultats que vous avez obtenus. Les éléments suivants doivent y apparaitre (cela peut vous servir de plan, mais ce n'est pas rigide) :

- Titre du projet, et nom de l'ensemble des membres de l'équipe (avec mail et matricule)
    
- **Introduction** : résumé du problème, de la méthodologie et des résultats obtenus.

- **Présentation du dataset** : description, justification de la taille, du choix des features, etc. 

- **Preprocessing** : s'il y en a, justification des étapes de preprocessing.

- **Methodologie** : description et justification de l'ensemble des choix (algorithmes, hyper-paramètres, régularisation, métriques, etc.)

- **Résultats** : analyse des résultats obtenus (utilisez des figures pour illustrer), mise en relation entre les choix de design et la performance obtenue.

- **Discussion** : discutez des avantages et des inconvénients de votre approche; quels sont les faiblesses, les failles ? Qu'est-ce qu'il peut être amélioré ? Vous pouvez également suggérer des futures idées d'exploration.

- **Références** : si vous vous êtes inspiré d'une étude déjà faite.
    
Vous pouvez utiliser le template d'arXiv pour le rapport : https://fr.overleaf.com/latex/templates/style-and-template-for-preprints-arxiv-bio-arxiv/fxsnsrzpnvwc. **L'ensemble du rapport ne doit cependant pas excéder 5 pages, figures et références compris.** Les 5 pages ne sont pas obligatoires, si vous estimez que moins est suffisant et que votre rapport est effectivement complet, vous ne serez pas pénalisé.


### 3. Rendu attendu

A la fin du TP, vous soumettrez un fichier *zip* contenant les éléments suivants:

- Le fichier *pdf* du rapport
- Ce notebook que vous aurez complété. Vous pouvez également implémenter votre méthode à la suite ici, ou alors utiliser un autre fichier si vous le souhaitez. Bien que seul le rapport servira pour la notation, ayez un code commenté et clair !
- Ne pas envoyer les fichiers de données, car trop conséquent. Avec le rapport et le code, tout sera détaillé et il sera possible de les refaire facilement.

### 4. Evalutation

12 points de cette partie sera basé sur la méthodologie, et 4 points sur les résultats.

La notation sur la méthodologie inclus : 

- La pertinence de l'ensemble des étapes de l'approche

- La bonne description des algorithmes choisis

- La justification judicieuse des choix établis

- Une analyse pertinente des résultats

- La clarté et l'organisation du rapport (figures, tables) et du code.


Pour ce qui est des résultats, il est impossible de mettre un barème fixe car ils vont dépendre du sujet que vous allez choisir. C'est un problème auquel vous serez confrontés : chaque étude étant spécifique, il peut être compliqué d'évaluer qualitativement un modèle, d'autant que vous n'avez sans doute pas connaissance de l'état de l'art. C'est pourquoi il va être important de faire plusieurs essais, et de comparer différentes méthodes. Ainsi, les résultats doivent être cohérent avec la complexité de votre implémentation : un modèle simple et naïf vous fournira des premiers résultats, que vous devrez ensuite améliorer avec des modèles plus précis et complexes.

De ce fait, l'ensemble des points pour les résultats seront donnés si : 
 - Vous obtenez des premiers résultats avec une méthode naïve qui témoignent de la pertinence de vos choix 
 - Ces résultats sont ensuite améliorés avec une méthode plus complexe
 

# Choix de l'équipe
Nous avons choisi le troisième sujet, soit *Méthodes de clusterings appliqué au tweet sur l'actualité, et analyse des résultats*. 

Tous nos résultats ont été générés dans des *notebooks* à part. 

Voir les autres fichiers du zip ainsi que le rapport en format pdf.

Le rapport en format pdf respecte le maximum de 5 pages et contient toutes les résultats pertinents pour comprendre le travail accompli lors de ce tp. Pour avoir plus de détails ou pour voir des résultats supplémentaires, vous pouvez consulter  le fichier d'annexe. 